# Soal nomor 1, 2502041956 - Nicholas Javier - LA05
# Link Video : https://www.youtube.com/watch?v=yF9EqOwTXt0

1. [LO1, LO2, LO3] Hate speech pada media sosial adalah suatu ungkapan yang sangat mengganggu.
Identifikasi hate speech secara otomatis akan membantu pihak terkait untuk melakukan suatu tindakan
secara efisien. Oleh sebab itu diperlukan bantuan dari anda untuk membangun model klasifikasi
multilabel Hate Speech pada data yang diberikan, dengan label Hate Speech (HS), Abusive,
HS_Individual, HS_Group, HS_Religion, HS_Race, HS_Physical, HS_Gender, HS_Other.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import re
import string
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
!pip install datasets
from datasets import Dataset

In [2]:
df = pd.read_csv('data_1C.csv')
df.head()

,Unnamed: 0,index,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,0,13012,Subhan?Allah Kenangan yg Tidak akan Terlupakan...,1,0,1,0,0,0,0,0,1
1,1,5594,"Direksi Pertamina Dirombak, ESDM Mau Premium t...",0,0,0,0,0,0,0,0,0
2,2,347,"3. Karena berbasis teknologi digital, Presiden...",0,0,0,0,0,0,0,0,0
3,3,4356,RT USER: USER USER USER kebanyakan alasan...\n...,1,1,1,0,0,0,0,1,0
4,4,10958,Ganyang PKI dg segala anteknya... URL,1,0,0,1,0,1,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3292 entries, 0 to 3291
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3292 non-null   int64 
 1   index          3292 non-null   int64 
 2   Tweet          3292 non-null   object
 3   HS             3292 non-null   int64 
 4   Abusive        3292 non-null   int64 
 5   HS_Individual  3292 non-null   int64 
 6   HS_Group       3292 non-null   int64 
 7   HS_Religion    3292 non-null   int64 
 8   HS_Race        3292 non-null   int64 
 9   HS_Physical    3292 non-null   int64 
 10  HS_Gender      3292 non-null   int64 
 11  HS_Other       3292 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 308.8+ KB


In [4]:
df.describe()

,Unnamed: 0,index,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
count,3292.00000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000
mean,1645.50000,6647.773390,0.425881,0.384569,0.277947,0.147934,0.057412,0.036148,0.024605,0.025213,0.293135
std,950.46287,3805.252449,0.494551,0.486567,0.448055,0.355089,0.232663,0.186687,0.154942,0.156794,0.455269
min,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,822.75000,3351.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1645.50000,6745.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2468.25000,9924.500000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,3291.00000,13168.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df.isnull().sum()

Unnamed: 0       0
index            0
Tweet            0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
dtype: int64

A. Bangunlah model klasifikasi dengan menggunakan Large Language Model (LLM), anda
dapat menggunakan pretrained model yang akan anda jelaskan pada penjelasan dengan
melakukan minimal 2 hyperparameter tuning.  

In [6]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    return text

df['Tweet'] = df['Tweet'].apply(preprocess_text)

labels = ['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']
df['labels'] = df[labels].values.tolist()

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_data['labels'])
val_labels = mlb.transform(val_data['labels'])
test_labels = mlb.transform(test_data['labels'])

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['Tweet'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], output_all_columns=True)
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], output_all_columns=True)
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], output_all_columns=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2369 [00:00<?, ? examples/s]

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

Map:   0%|          | 0/659 [00:00<?, ? examples/s]

Untuk teks dalam dataset df, kode melakukan preprocessing, pembagian data, dan tokenisasi. Pertama, fungsi preprocess_text membersihkan teks dengan menghapus URL, tanda baca, dan mengubah teks menjadi huruf kecil. Fungsi ini kemudian diterapkan pada kolom "Tweet" dalam dataframe. Selanjutnya, kolom yang disebut "label" diisi dengan daftar label yang terdapat di setiap baris. Dengan menggunakan train_test_split, data dibagi menjadi set pelatihan, validasi, dan pengujian. Dataset ini kemudian diformat seperti dataset library. Untuk mengubah label teks menjadi format biner, multilabel binarizer digunakan. Selanjutnya, teks diubah menjadi token yang dapat dimasukkan ke dalam model dengan tokenizer model DistilBert. Tokenisasi dilakukan pada ketiga set data: pelatihan, validasi, dan pengujian. Selanjutnya, set data diformat menggunakan PyTorch dengan kolom output utama input_ids dan attention_mask.

In [7]:
!pip install transformers[torch] accelerate -U

Kode dibawah ini mendefinisikan model klasifikasi sekuens multi-label berbasis DistilBert dan melatihnya menggunakan Trainer transformer library. Pertama, definisi kelas DistilBertForSequenceClassification dengan mewarisi dan mengganti lapisan klasifikasinya. Fungsi maju diubah untuk menangani tugas klasifikasi berbagai label menggunakan BCEWithLogitsLoss. Setelah itu, model diaktifkan dengan memuat parameter pra-terlatih dari distilbert-base-uncased dan mengubah jumlah label sesuai dengan jumlah kelas dalam dataset. Pengaturan direktori keluaran, strategi evaluasi, laju pembelajaran, ukuran batch, jumlah epoch, dan pengurangan berat adalah argumen pelatihan. Model, argumen pelatihan, dataset evaluasi, dan tokenizer kemudian digunakan untuk menginisialisasi instruktur. Terakhir, model dilatih menggunakan metode pelatihan pelatih, yang secara otomatis mengelola proses pelatihan.

In [14]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch.nn as nn

class DistilBertForMultiLabelSequenceClassification(DistilBertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs[0]
        pooled_output = hidden_state[:, 0]
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
        output = (logits,) + outputs[1:]
        return ((loss,) + output) if loss is not None else output

model = DistilBertForMultiLabelSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(labels))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.366580
2,No log,0.335563
3,No log,0.330580
4,0.328900,0.308456
5,0.328900,0.304655


TrainOutput(global_step=745, training_loss=0.29505649029008496, metrics={'train_runtime': 169.6674, 'train_samples_per_second': 69.813, 'train_steps_per_second': 4.391, 'total_flos': 422960655897252.0, 'train_loss': 0.29505649029008496, 'epoch': 5.0})

In [15]:
from sklearn.metrics import classification_report

train_preds = trainer.predict(train_dataset)
train_pred_labels = (train_preds.predictions > 0).astype(int)

val_preds = trainer.predict(val_dataset)
val_pred_labels = (val_preds.predictions > 0).astype(int)

test_preds = trainer.predict(test_dataset)
test_pred_labels = (test_preds.predictions > 0).astype(int)

train_report = classification_report(train_preds.label_ids, train_pred_labels, target_names=labels)
val_report = classification_report(val_preds.label_ids, val_pred_labels, target_names=labels)
test_report = classification_report(test_preds.label_ids, test_pred_labels, target_names=labels)

print("Training Set Classification Report:")
print(train_report)

print("Validation Set Classification Report:")
print(val_report)

print("Testing Set Classification Report:")
print(test_report)


Training Set Classification Report:
               precision    recall  f1-score   support

           HS       0.93      0.91      0.92      1009
      Abusive       0.93      0.83      0.88       920
HS_Individual       0.78      0.75      0.76       657
     HS_Group       0.73      0.45      0.55       352
  HS_Religion       0.85      0.21      0.33       136
      HS_Race       0.88      0.37      0.52        75
  HS_Physical       0.00      0.00      0.00        59
    HS_Gender       0.00      0.00      0.00        62
     HS_Other       0.81      0.82      0.82       701

    micro avg       0.87      0.75      0.80      3971
    macro avg       0.66      0.48      0.53      3971
 weighted avg       0.84      0.75      0.78      3971
  samples avg       0.45      0.41      0.42      3971

Validation Set Classification Report:
               precision    recall  f1-score   support

           HS       0.78      0.68      0.73       107
      Abusive       0.76      0.73      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Menurut laporan klasifikasi untuk set validasi dan pengujian, model menunjukkan kinerja yang baik pada beberapa kelas, seperti kelas "HS" dan "Abusive", yang memiliki nilai precision, recall, dan f1-score yang relatif tinggi. Namun, kinerja model yang sangat rendah pada kelas lain, seperti "HS_Physical", "HS_Gender", dan "HS_Religion", menunjukkan bahwa model kesulitan Singkatnya, model ini dapat menemukan beberapa jenis ujaran kebencian dan konten pelecehan dengan cukup baik. Ini terutama berlaku untuk kelas dengan data yang lebih kecil.

In [16]:
from sklearn.metrics import classification_report, accuracy_score

train_accuracy = accuracy_score(train_preds.label_ids, train_pred_labels)
val_accuracy = accuracy_score(val_preds.label_ids, val_pred_labels)
test_accuracy = accuracy_score(test_preds.label_ids, test_pred_labels)

print(f"Testing Set Accuracy: {test_accuracy:.2f}")

Testing Set Accuracy: 0.53


Saya melakukan hyperparameter tuning untuk soal training dibawah dengan mengubah epoch, learning rate, dan batch size.

In [17]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=7,
    weight_decay=0.01,
)


trainer_hyper = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer_hyper.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.294904
2,No log,0.308580
3,No log,0.332695
4,No log,0.326388
5,No log,0.326394
6,No log,0.340365
7,0.141200,0.333593


TrainOutput(global_step=525, training_loss=0.138319027310326, metrics={'train_runtime': 201.6882, 'train_samples_per_second': 82.221, 'train_steps_per_second': 2.603, 'total_flos': 592147298826864.0, 'train_loss': 0.138319027310326, 'epoch': 7.0})

In [18]:
trainh_preds = trainer_hyper.predict(train_dataset)
trainh_pred_labels = (trainh_preds.predictions > 0).astype(int)

valh_preds = trainer_hyper.predict(val_dataset)
valh_pred_labels = (valh_preds.predictions > 0).astype(int)

testh_preds = trainer_hyper.predict(test_dataset)
testh_pred_labels = (testh_preds.predictions > 0).astype(int)

trainh_report = classification_report(trainh_preds.label_ids, trainh_pred_labels, target_names=labels)
valh_report = classification_report(valh_preds.label_ids, valh_pred_labels, target_names=labels)
testh_report = classification_report(testh_preds.label_ids, testh_pred_labels, target_names=labels)

print("Training Set Classification Report:")
print(trainh_report)

print("Validation Set Classification Report:")
print(valh_report)

print("Testing Set Classification Report:")
print(testh_report)



Training Set Classification Report:
               precision    recall  f1-score   support

           HS       1.00      0.99      1.00      1009
      Abusive       0.99      0.97      0.98       920
HS_Individual       0.96      0.95      0.95       657
     HS_Group       0.91      0.91      0.91       352
  HS_Religion       0.96      0.87      0.91       136
      HS_Race       0.91      0.77      0.83        75
  HS_Physical       0.89      0.14      0.24        59
    HS_Gender       0.83      0.79      0.81        62
     HS_Other       0.97      0.99      0.98       701

    micro avg       0.97      0.95      0.96      3971
    macro avg       0.93      0.82      0.85      3971
 weighted avg       0.97      0.95      0.95      3971
  samples avg       0.53      0.52      0.53      3971

Validation Set Classification Report:
               precision    recall  f1-score   support

           HS       0.78      0.74      0.76       107
      Abusive       0.80      0.73      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [19]:
from sklearn.metrics import classification_report, accuracy_score

trainh_accuracy = accuracy_score(trainh_preds.label_ids, trainh_pred_labels)
valh_accuracy = accuracy_score(valh_preds.label_ids, valh_pred_labels)
testh_accuracy = accuracy_score(testh_preds.label_ids, testh_pred_labels)

print(f"Testing Set Accuracy: {testh_accuracy:.2f}")

Testing Set Accuracy: 0.57


Laporan klasifikasi untuk set pelatihan menunjukkan kinerja yang sangat baik untuk model, dengan nilai precision, recall, dan f1-score yang tinggi untuk hampir semua kelas. Ini menunjukkan bahwa model mampu mengenali dan mengklasifikasikan data pelatihan dengan sangat akurat. Namun, performa model menurun secara signifikan pada set validasi dan pengujian, terutama pada kelas dengan jumlah data yang lebih sedikit seperti "HS_Physical" dan "HS_Gender".

Dibandingkan dengan penelitian sebelumnya, terlihat bahwa model menghadapi masalah yang serupa dalam mengklasifikasikan beberapa kelas yang sama pada set pengujian dan validasi. Misalnya, kelas "HS_Physical" dan "HS_Gender" menunjukkan skor f1 yang sangat rendah atau nol di kedua laporan. Namun, kelas lain, seperti "HS_Religion" dan "HS_Race", menunjukkan peningkatan kinerja pada set pengujian dan data pelatihan, yang menunjukkan bahwa model mungkin membutuhkan penyesuaian tambahan, mungkin dengan menambahkan lebih banyak data untuk kelas-kelas yang lebih kecil
